# Création DataFrame IIT_CDIP_IND_XML_FEATURES

## README
Ce notebook permet d'exploiter les fichiers xml individuels des fichiers présents sur la base de données IIT-CDIP, et de rassembler les informations extraites dans le DataFrame iit_cdip_ind_features

Il réalise tout d'abord certaines opérations préalables (chapitre 1), dont la définition des variables globales d'exécution (**A METTRE A JOUR LORS D'UNE PREMIERE UTILISATION**)

A l'issue (chapitre 2), il permet de créer le DataFrame, qui contient les informations suivantes:
- document_id
- iit_individual_xml_path
- bt: non encore déterminé
- title
- document_date
- dt: non encore déterminé
- author
- pages_amount
- scan_date
- ocr_text
- np: non encore déterminé
- ca: non encore déterminé
- no: non encore déterminé
- fn: non encore déterminé
- trc: non encore déterminé
- bx: non encore déterminé
- pc: non encore déterminé
- cr: non encore déterminé

## 1. Préparation

In [ ]:
import os
import time
import numpy as np
import pandas as pd

from lxml import etree
from functools import reduce
from utils import remove_ds_store_files

In [ ]:
project_path = '/Users/ben/Work/mle/ds-project/mai25_bds_extraction/' # à modifier par chacun en fonction de son arborescence

data_path = os.path.join(project_path, 'data')
raw_data_path = os.path.join(data_path, 'raw')
processed_data_path = os.path.join(data_path, 'processed')

raw_rvl_cdip_path = os.path.join(raw_data_path, 'RVL-CDIP')
rvl_cdip_images_path = os.path.join(raw_rvl_cdip_path, 'images')
rvl_cdip_labels_path = os.path.join(raw_rvl_cdip_path, 'labels')

iit_cdip_images_path = os.path.join(raw_data_path, 'IIT-CDIP', 'images')
iit_cdip_xmls_path = os.path.join(raw_data_path, 'IIT-CDIP', 'xmls')

## 2. Création du DataFrame IIT_CDIP_IND_XML_FEATURES

## 2.1. Création de la base de la DataFrame

In [ ]:
df_documents = pd.read_parquet(os.path.join(processed_data_path, "df_documents.parquet"))
df_documents.head()

Seule une partie des documents possèdent un fichier xml individuel. On sélectionne donc les lignes concernées avant de procéder à l'extraction des features dans les xml.

In [ ]:
df_base = df_documents[~df_documents.iit_individual_xml_path.isna()][["document_id", "iit_individual_xml_path"]]

In [ ]:
len(df_base)

In [ ]:
df_base.head()

## 2.2. Extraction des features des xml

In [ ]:
tags = []
parser = etree.XMLParser(recover=True, encoding="ISO-8859-1")
for index, row in df_base.iterrows():
    document_id, relative_path = row.values.tolist()
    filename = os.path.join(data_path, relative_path)
    try:
        tree = etree.parse(filename, parser)
        root = tree.getroot()
        for element in root.iterchildren(tag=etree.Element):
            if element.tag not in tags:
                tags.append(element.tag)
    except Exception as e:
        print(f"Erreur avec le fichier {filename}")
print(tags)

A partir de cette liste de tags, on créer un dictionnaire de correspondance. Il nous servira, en fin de processus d'extraction à renommer les colonnes de la DataFrame. Les correspondances ont été établies en analysant les données contenues par les différents tags.

Extrait du README qui accompagne les fichiers texte de IIT-CDIP:

*The records contain both text and metadata. The text was produced by applying optical character recognition (OCR) to document images in TIFF format. The metadata was produced by the tobacco organizations using a variety of techniques. It includes a title, a listing of the senders and recipients of the document, important names mentioned in the document, controlled vocabulary categories, geographical and organizational context data, and other information. Not all metadata fields are available for all documents, the formatting is inconsistent, and there is an unknown level of errors and omissions.*

In [ ]:
parsed_fields = {
    'tid': 'document_id',
    'bt': 'bt',
    'ti': 'title',
    'dd': 'document_date',
    'dt': 'dt',
    'au': 'author',
    'pg': 'pages_amount',
    'dl': 'scan_date',
    'ot': 'ocr_text',
    'np': 'np',
    'ca': 'ca',
    'no': 'no',
    'fn': 'fn',
    'rc': 'trc',
    'bx': 'bx',
    'pc': 'pc',
    'cr': 'cr'
}

In [ ]:
def extract_info_from_xmls():
    parser = etree.XMLParser(recover=True, encoding="ISO-8859-1")
    tmp_list = []
    for index, row in df_base.iterrows():
        document_id, relative_path = row.values.tolist()
        filename = os.path.join(data_path, relative_path)
    #    print(filename)
        try:
            tree = etree.parse(filename, parser)
            root = tree.getroot()
            tmp_list.append({
                key: root.findtext(key) if root.findtext(key) is not None else np.nan
                    for key in parsed_fields})
        except Exception as e:
            print(f"Erreur avec le fichier {filename}")
    df = pd.DataFrame(tmp_list)
    df.rename(columns = parsed_fields, inplace=True)
    return df


In [ ]:
t = time.time()
df_xmls = extract_info_from_xmls()
print(f"Duree d'exécution: {time.time() - t:.3f} secondes.")
df_xmls.head()

## 2.3. Création de la DataFrame et sauvegarde

In [ ]:
df_iit_cdip_ind_xml_features = df_base.merge(df_xmls, on="document_id", how="left")
df_iit_cdip_ind_xml_features.head()

In [ ]:
df_iit_cdip_ind_xml_features.to_parquet(os.path.join(processed_data_path, "df_iit_cdip_ind_xml_features.parquet"))